# FE 1

In [1]:
# imports
import pandas as pd
# import sys
# print(f"Running Python: {sys.executable}")
# import os
# print(os.getcwd())
import warnings
warnings.filterwarnings('ignore')


In [2]:
# custom packages imports
import get_data as gd
import preprocessing
from parameters import PARAM 


In [3]:
import torch
from torch.utils.data import Dataset
from transformers import LongformerTokenizer, LongformerForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import accuracy_score, f1_score

In [4]:
dir(preprocessing)

['PARAM',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'get_exploded_df',
 'get_label_encoding']

In [5]:
df_train, df_test = gd.get_data()
#df = preprocessing.get_exploded_df(df_train)
df = preprocessing.get_label_encoding(df_train)

# Aproach1: Conversational Modeling with Entire Dialogues

likely the most straightforward and effective method.

- Context Preservation: The entire conversation is fed into the model, maintaining the sequence and flow of dialogue.
- Simplicity: While computationally intensive, it is simpler to implement compared to hierarchical or Siamese models.
- Effectiveness: Transformer models excel at capturing long-range dependencies in text.

## Data Preparation

In [6]:
#df = df.dropna(subset=['response_a', 'response_b'])

In [7]:
special_tokens = {
    'additional_special_tokens': ['[USER]', '[MODEL_A]', '[MODEL_B]']
}

In [8]:
import json

def concatenate_conversation(row):
    # Parse the string representations into lists using json.loads
    prompts = json.loads(row['prompt'])
    responses_a = json.loads(row['response_a'])
    responses_b = json.loads(row['response_b'])
    
    # Ensure all lists are of the same length
    min_length = min(len(prompts), len(responses_a), len(responses_b))
    prompts = prompts[:min_length]
    responses_a = responses_a[:min_length]
    responses_b = responses_b[:min_length]
    
    conversation = ""
    for prompt, resp_a, resp_b in zip(prompts, responses_a, responses_b):
        # Handle None values
        prompt_text = '' if prompt is None else prompt.strip()
        resp_a_text = '' if resp_a is None else resp_a.strip()
        resp_b_text = '' if resp_b is None else resp_b.strip()
        
        conversation += f"[USER]: {prompt_text}\n[MODEL_A]: {resp_a_text}\n[MODEL_B]: {resp_b_text}\n"
    return conversation



In [9]:
df['conversation'] = df.apply(concatenate_conversation, axis=1)

In [10]:
df['conversation'][0]

'[USER]: Is it morally right to try to have a certain percentage of females on managerial positions?\n[MODEL_A]: The question of whether it is morally right to aim for a certain percentage of females in managerial positions is a complex ethical issue that involves considerations of fairness, equality, diversity, and discrimination.\n\nHere are some arguments in favor of and against such policies:\n\n**Arguments in favor:**\n\n1. **Correcting Historical Inequities:** Women have historically been underrepresented in leadership roles due to various cultural, institutional, and social barriers. Aiming for a specific percentage can be seen as a corrective measure to address past and ongoing discrimination.\n\n2. **Promoting Diversity:** Diverse leadership teams can enhance decision-making and represent a broader range of perspectives. This can lead to better outcomes for organizations and society as a whole.\n\n3. **Equality of Opportunity:** Setting targets for female representation in man